# Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

#### TODOS
1. DONE Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
2. try on leaderboard
3. Include Odometry and fuse into heads
    - Speed
    - Location
4. navigation
5. controller
6. Evaluation on Test set, Modularization



## Dependencies

In [1]:
# MODEL STUFF
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# GENERAL STUFF
import time
import copy
import os
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline') # TODO

# DATA ENGINEERING
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing

## Model

In [2]:
class MyResnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # ResNet Architecture with pretrained weights, also bigger resnets available
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features

        # Top layer of ResNet which you can modify. We choose Identity to use it as Input for all the heads
        self.net.fc = nn.Identity()
        
        
        # Regression Heads for Throttle, Brake and Steering
        self.thr_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.str_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Tanh() # [-1,1] Range Output
        )

    # Forward Pass of the Model
    def forward(self, x):
        x = self.net(x)
        #x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x) # 3 Outputs since we have 3 Heads

## Data Loaders, Data Sets

In [3]:
#path_ege_data = os.path.join("..", "..", "data", "Dataset Ege")
train_path = "D:/data/Train"
test_path = "D:/data/Test"

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=train_path, config=config)
test_dataset = CARLADataset(root_dir=test_path, config=config)

weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
len(train_dataloader)

1367

In [5]:
len(test_dataloader)

372

In [6]:
mean = torch.tensor([79.6657, 81.5673, 105.6161])
std = torch.tensor([66.8309, 60.1001, 66.2220])


transform_norm = transforms.Compose([
    transforms.Normalize(mean, std)
])


## Training

In [7]:
# Initialise Model (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet().cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [8]:
def to_cuda_if_possible(data):
    return data.to(device) if device else data

In [12]:
def forward_pass(data):
    # further preprocessing
    
    X_rgb = torch.squeeze(transform_norm(data["rgb"])).float()
    #print(X_rgb.shape)
    #print(X_rgb[0,0,0,0])
    X_throttle = data["throttle"].float()
    X_steer = data["steer"].float()
    X_brake = data["brake"].float()

    # move to GPU
    X_rgb = to_cuda_if_possible(X_rgb)
    X_throttle = to_cuda_if_possible(X_throttle)
    X_steer = to_cuda_if_possible(X_steer)
    X_brake = to_cuda_if_possible(X_brake)

    # compute outputs
    optimizer.zero_grad()

    Y_hat = net(X_rgb)
    Y_hat_throttle = to_cuda_if_possible(Y_hat[0])
    Y_hat_steer = to_cuda_if_possible(Y_hat[1])
    Y_hat_brake = to_cuda_if_possible(Y_hat[2])

    # get labels from data
    Y_throttle = to_cuda_if_possible(data["throttle"].float())
    Y_steer = to_cuda_if_possible(data["steer"].float())
    Y_brake = to_cuda_if_possible(data["brake"].float())

    # Calculate Loss
    loss_throttle = criterion(Y_hat_throttle, Y_throttle)
    loss_steer = criterion(Y_hat_steer, Y_steer)
    loss_brake = criterion(Y_hat_brake, Y_brake)
    loss = sum([loss_throttle, loss_steer, loss_brake])/3
    return loss

In [13]:
# Loss and Optimizer
criterion = nn.L1Loss() # Easy to interpret #nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001) #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Model Trainer Prototype

In [15]:
%%time

n_epochs = 1
print_every = 100
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader): #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])

        loss = forward_pass(data)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (batch_idx) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0

    
    # Evaluation on Test set, skipped for now

    with torch.no_grad():
        net.eval()
        
        for batch_idx, data in enumerate(test_dataloader):
            
            loss = forward_pass(data)
            
            batch_loss += loss.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')

    # Back to training
    net.train()
    
    

Epoch 1



ValueError: cannot reshape array of size 29491200 into shape (160,960,3)

### Test predictions

In [14]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(next(iter(test_dataloader)).keys())
idx, data = next(enumerate(test_dataloader))

dict_keys(['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])


In [11]:
data_ = to_cuda_if_possible(torch.squeeze(transform_norm(data["rgb"]),1).float())
target_ = (data["throttle"],data["steer"],data["brake"])


#print(target_[0].shape)

In [12]:
with torch.no_grad():
    net.eval()
    outputs_ = net(data_)

In [13]:
i =0

In [14]:
print(np.round(outputs_[i].cpu().numpy(),1))

[[0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.4]
 [0.3]
 [0.3]
 [0.4]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.4]
 [0.3]
 [0.4]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.4]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.4]
 [0.3]
 [0.3]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.3]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.3]
 [0.3]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.3]
 [0.4]
 [0.4]]


In [15]:
print(np.round(target_[i].cpu().numpy(),1))

[[0.8]
 [0.8]
 [0.8]
 [0.8]
 [0.8]
 [0.8]
 [0.1]
 [0.8]
 [0.4]
 [0.1]
 [0.8]
 [0. ]
 [0.1]
 [0.8]
 [0.1]
 [0.8]
 [0.8]
 [0.1]
 [0.8]
 [0. ]
 [0.1]
 [0.8]
 [0.1]
 [0.8]
 [0.8]
 [0.1]
 [0.8]
 [0. ]
 [0.2]
 [0.8]
 [0.1]
 [0.8]
 [0.8]
 [0.1]
 [0.8]
 [0.3]
 [0. ]
 [0.8]
 [0.1]
 [0.7]
 [0.8]
 [0.1]
 [0.8]
 [0.6]
 [0.1]
 [0.8]
 [0. ]
 [0. ]
 [0.8]
 [0.1]
 [0.5]
 [0.8]
 [0.1]
 [0.8]
 [0.7]
 [0.1]
 [0.8]
 [0.1]
 [0.1]
 [0.8]
 [0.1]
 [0.8]
 [0.8]
 [0.1]]


In [16]:
np.mean(abs(outputs_[i].cpu().numpy()-target_[i].cpu().numpy()))

0.33136968132553096

### IMG Processing

BGR is now standard FOR carla agent and training

In [19]:
import cv2
idx, batch = next(enumerate(test_dataloader))
print(batch["rgb"].shape)

torch.Size([64, 1, 3, 160, 960])


In [52]:
img = batch["rgb"][0]#.shape
img = img.numpy().astype(np.uint8).reshape(160,960,3)
print(img.shape)

#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # NUR HIER, NICHT IN CARLA AGENT
print(img.shape)
print(type(img))
transform = transforms.Compose([transforms.ToPILImage()])

tensor = transform(img)

#print(type(tensor))

tensor.show()

#torch.tensor(tensor)

(160, 960, 3)
(160, 960, 3)
<class 'numpy.ndarray'>


TEST

In [49]:
tensor = transform_norm(torch.squeeze(data["rgb"],1))

In [47]:
tensor = torch.squeeze(transform_norm(data["rgb"])).float()

In [50]:
for i in range(64):
    print(np.mean(tensor.numpy()[i], axis = (1,2)))

[1.54852209 0.88420938 1.37634886]
[1.10096353 0.55371714 0.59199937]
[-1.08535938 -0.99923625 -1.02566063]
[1.13848369 0.234791   0.78299234]
[-1.06880797 -1.01068752 -1.14102454]
[0.7334713  0.33303551 0.28869145]
[-1.15103684 -1.23976422 -1.43820144]
[-1.14608167 -1.22912745 -1.40241086]
[0.91805144 0.8640672  0.78341616]
[-1.13446633 -1.24165754 -1.39148279]
[ 0.85121502  0.63044785 -0.01603827]
[-0.3211704   0.10443372 -0.44987601]
[0.91957532 0.53315758 0.7227442 ]
[-1.1554799  -1.30859979 -1.4596305 ]
[-1.1460011  -1.29597241 -1.44079874]
[0.67091486 0.42453769 0.27095941]
[1.00636172 0.58392282 0.52279589]
[0.96530203 0.3602333  0.72535349]
[0.56183388 0.5095221  0.50605621]
[ 0.39433373  0.11186631 -0.30136611]
[0.79173596 0.2989251  0.72712793]
[0.42177443 0.27534346 0.56471357]
[0.35257517 0.21272993 0.01800983]
[0.97932656 0.37091741 0.51793623]
[0.75757783 0.33220259 0.68871586]
[ 0.44497771 -0.02873148 -0.0232817 ]
[-1.1330878  -1.27234203 -1.47967091]
[0.47495328 0.39759

### Vanishing/Exploding Gradients

In [ ]:
"""
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, param.data.cpu().numpy())
"""

In [ ]:
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, np.max(abs(param.data.cpu().numpy())))

## Saving and Loading

Not suited for leaderboard agents

In [ ]:
#torch.save(net, 'rgb_resnet.pth')

In [ ]:
#net = torch.load('rgb_resnet.pth')

suited for leaderboard agents

In [ ]:
#torch.save(net.state_dict(), "rgb_resnet.pth")

In [ ]:
#net = MyResnet()
#net.load_state_dict(torch.load("rgb_resnet.pth"))
#net.cuda()

## Testing Time

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
idx, X = next(enumerate(test_dataloader))
img = transform_norm(X["rgb"])
img.shape

In [ ]:
torch.squeeze(img,1).shape

In [ ]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 37 sekunden ~2gb
# 24.12: 44 batches -> preprocessing 26 sec, training & preprocessing 69 sec
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
print(len(test_dataloader))
at = time.time()
for batch_idx, data in enumerate(test_dataloader):
    #print(batch_idx)
    data_ = torch.squeeze(transform_norm(data["rgb"]),1).float()
    #print(data_.shape)
et = time.time()
print(et-at)
